# compare the OpenPIV Python with PIVLab


In [1]:
%reload_ext watermark
%watermark -v -m -p numpy,openpiv

Python implementation: CPython
Python version       : 3.8.5
IPython version      : 7.19.0

numpy  : 1.19.4
openpiv: 0.23.3b0

Compiler    : GCC 7.3.0
OS          : Linux
Release     : 5.4.0-58-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 8
Architecture: 64bit



In [2]:
from openpiv import windef

import numpy as np
import os
from time import time

import matplotlib.pyplot as plt
%matplotlib tk

import matplotlib
matplotlib.rcParams['figure.figsize'] = (8.0, 6.0)


In [3]:
settings = windef.Settings()

# 'Data related settings'
# Folder with the images to process
settings.filepath_images = '../data/'
# Folder for the outputs
settings.save_path = '../OpenPIV_results/'
# Root name of the output Folder for Result Files
settings.save_folder_suffix = 'Test_1'
# Format and Image Sequence
settings.frame_pattern_a = 'karman_16Hz_000_A.jpg'
settings.frame_pattern_b = 'karman_16Hz_000_B.jpg'

'Region of interest'
# (50,300,50,300) #Region of interest: (xmin,xmax,ymin,ymax) or 'full' for full image
# settings.ROI = 'full'
# settings.ROI = (200,400,500,900)
# settings.ROI = (200,212,500,512)

settings.deformation_method = 'symmetric' 
#settings.deformation_method = 'second image'


settings.num_iterations = 4  # select the number of PIV passes

# add the interrogation window size for each pass. 
# For the moment, it should be a power of 2 
settings.windowsizes=(64, 32, 16, 6)
settings.overlap=(32, 16, 8, 3)

# settings.windowsizes = (128, 64, 32, 16, 8) # if longer than n iteration the rest is ignored
# The overlap of the interroagtion window for each pass.
# settings.overlap = (64, 32, 16, 8, 4) # This is 50% overlap


# Has to be a value with base two. In general window size/2 is a good choice.
# methode used for subpixel interpolation: 'gaussian','centroid','parabolic'
settings.subpixel_method = 'gaussian'

# order of the image interpolation for the window deformation
settings.interpolation_order = 3
settings.scaling_factor = 1  # scaling factor pixel/meter
settings.dt = 1  # time between to frames (in seconds)

# 'Signal to noise ratio options (only for the last pass)'
# It is possible to decide if the S/N should be computed (for the last pass) or not
# settings.extract_sig2noise = True  # 'True' or 'False' (only for the last pass)
settings.sig2noise_threshold = 1.0
# method used to calculate the signal to noise ratio 'peak2peak' or 'peak2mean'
settings.sig2noise_method = 'peak2peak'
# select the width of the masked to masked out pixels next to the main peak
settings.sig2noise_mask = 2
settings.sig2noise_validate = False

# If extract_sig2noise==False the values in the signal to noise ratio
# output column are set to NaN

# only effecting the first pass of the interrogation the following passes
# in the multipass will be validated

'Output options'
# Select if you want to save the plotted vectorfield: True or False
settings.save_plot = False
# Choose wether you want to see the vectorfield or not :True or False
settings.show_plot = False
settings.scale_plot = 100  # select a value to scale the quiver plot of the vectorfield
# run the script with the given settings



# 'Processing Parameters'
settings.correlation_method='circular'  # 'circular' or 'linear'
settings.normalized_correlation = False

# 'vector validation options'
# choose if you want to do validation of the first pass: True or False
settings.validation_first_pass = True


settings.filter_method = 'localmean'


settings.replace_vectors = True
# maximum iterations performed to replace the outliers
settings.max_filter_iteration = 10
settings.filter_kernel_size = 2  # kernel size for the localmean method



settings.MinMax_U_disp = (-10, 10)
settings.MinMax_V_disp = (-10, 10)

# The second filter is based on the global STD threshold
settings.std_threshold = 5  # threshold of the std validation

# The third filter is the median test (not normalized at the moment)
settings.median_threshold = 3  # threshold of the median validation
# On the last iteration, an additional validation can be done based on the S/N.
settings.median_size = 1 # defines the size of the local median, it'll be 3 x 3

# New settings for version 0.23.2c
settings.image_mask = True

# Image mask properties
settings.dynamic_masking_method = 'intensity'
settings.dynamic_masking_threshold = 0.1
settings.dynamic_masking_filter_size = 21



# Smoothing after the first pass
settings.smoothn = True #Enables smoothing of the displacemenet field
settings.smoothn_p = 0.75 # This is a smoothing parameter


settings.show_all_plots = False

In [15]:
import glob
file_list = sorted(glob.glob('../data/karman_16Hz_*.jpg'))
# file_list = file_list[-2:]
# ile_list[0].rsplit('/')[-1]
# print(file_list)

'karman_16Hz_000_A.jpg'

In [17]:
counter = 0
for f,g in zip(file_list[::2],file_list[1::2]):
    print(f,g)
    settings.frame_pattern_a = f.rsplit('/')[-1]
    settings.frame_pattern_b = g.rsplit('/')[-1]
    settings.save_folder_suffix = f'Test_{counter}'
    windef.piv(settings)
    counter += 1
    

../data/karman_16Hz_000_A.jpg ../data/karman_16Hz_000_B.jpg
global filter invalidated -- vectors
std filter invalidated -- vectors
median filter invalidated 1 vectors
global filter invalidated -- vectors
std filter invalidated -- vectors
median filter invalidated 0 vectors
global filter invalidated -- vectors
std filter invalidated -- vectors
median filter invalidated 2 vectors
global filter invalidated -- vectors
std filter invalidated -- vectors
median filter invalidated 1 vectors
Image Pair 1
../data/karman_16Hz_001_A.jpg ../data/karman_16Hz_001_B.jpg
global filter invalidated -- vectors
std filter invalidated -- vectors
median filter invalidated 1 vectors
global filter invalidated -- vectors
std filter invalidated -- vectors
median filter invalidated 1 vectors
global filter invalidated -- vectors
std filter invalidated -- vectors
median filter invalidated 7 vectors
global filter invalidated -- vectors
std filter invalidated -- vectors
median filter invalidated 1 vectors
Image Pair 

In [5]:
from pivpy import pivpy, io, graphics
import xarray as xr

In [30]:
import glob

file_list = sorted(glob.glob('../OpenPIV_results/Open_PIV_results_'+str(settings.windowsizes[settings.num_iterations-1])+'_Test_*/*.txt'))
file_list
file_list.append(file_list.pop(2))
file_list

['../OpenPIV_results/Open_PIV_results_6_Test_0/field_A000.txt',
 '../OpenPIV_results/Open_PIV_results_6_Test_1/field_A000.txt',
 '../OpenPIV_results/Open_PIV_results_6_Test_2/field_A000.txt',
 '../OpenPIV_results/Open_PIV_results_6_Test_3/field_A000.txt',
 '../OpenPIV_results/Open_PIV_results_6_Test_4/field_A000.txt',
 '../OpenPIV_results/Open_PIV_results_6_Test_5/field_A000.txt',
 '../OpenPIV_results/Open_PIV_results_6_Test_6/field_A000.txt',
 '../OpenPIV_results/Open_PIV_results_6_Test_7/field_A000.txt',
 '../OpenPIV_results/Open_PIV_results_6_Test_8/field_A000.txt',
 '../OpenPIV_results/Open_PIV_results_6_Test_9/field_A000.txt',
 '../OpenPIV_results/Open_PIV_results_6_Test_10/field_A000.txt']

In [31]:
def read_data(file_list):
    data = []
    frame = 0
    for f in file_list:
        data.append(io.load_txt(f,frame=frame))
        frame += 1

    data = xr.concat(data,dim='t')
    data.attrs['units']= ['pix','pix','pix/dt','pix/dt']
    return data
# file_list = sorted(glob.glob('../OpenPIV_results/Open_PIV_results_6_Test_1/*.txt'))

In [34]:
import matplotlib.pyplot as plt
def plot_data(data):
    fig, ax = plt.subplots(1,len(file_list),figsize=(10,50))
    # for ax in axs:
    for t in data.t.values:
        ax[t].quiver(data.x.data, data.y.data, data.u.isel(t=0).data.T, data.v.isel(t=0).data.T, color='r', scale=120)
        s = ax[t].pcolormesh(data.x,data.y,data.w.T.isel(t=0), shading='gouraud', vmin=-.3, vmax=.3,alpha=0.7)
        # ax.invert_yaxis()
        ax[t].set_aspect(1)
    fig.colorbar(s, ax=ax[t], orientation='horizontal')
    plt.show()

In [33]:
data = read_data(file_list)
data.piv.vorticity();
plot_data(data)